<a href="https://www.kaggle.com/code/marianameyerf/notebook40af724143?scriptVersionId=109429558" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BR_BERTo_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BR_BERTo

CENTENFolha - 8 partes

Batch size = 16

GPU T4x2

(07h:21m)

In [2]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cjv7y01t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cjv7y01t
  Resolved https://github.com/huggingface/transformers to commit 98c9c5add92aa66d9f09af192139b4cb502aa9fb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.24.0.dev0-py3-none-any.whl size=5429247 sha256=9693e66f7b4c1a3da43f751b857eaaab4296459d016b6ed36709b2869a03f74f
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrpqnajt/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Unin

In [3]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3724k  100 3724k    0     0  23.1M      0 --:--:-- --:--:-- --:--:-- 23.1M


In [4]:
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ac
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ad
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ae 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_af
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ag
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ah

--2022-10-28 17:16:50--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/plain]
Saving to: ‘cetenfolha_aa.2’

cetenfolha_aa.2     100%[===================>]   3.64M  --.-KB/s    in 0.07s   

2022-10-28 17:16:50 (54.6 MB/s) - ‘cetenfolha_aa.2’ saved [3813899/3813899]

--2022-10-28 17:16:51--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [te

In [5]:
!cat cetenfolha_aa cetenfolha_ab cetenfolha_ac cetenfolha_ad cetenfolha_ae cetenfolha_af cetenfolha_ag cetenfolha_ah>  dump.txt
!awk NF < dump.txt > kant.txt

In [6]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")

model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695M [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-fc54e5dd0afbe4cd/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
#@title Step 10: Building the Dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=32,
)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [10]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets["train"][4]

{'input_ids': [0,
  225,
  41,
  269,
  271,
  1075,
  16,
  262,
  6120,
  268,
  45211,
  4745,
  469,
  2781,
  935,
  639,
  225,
  18,
  225,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [12]:
#block_size = tokenizer.model_max_length
block_size = 32

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [15]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [16]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [17]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BR_BERTo-CENTENFolha-8_parts",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [18]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 219241
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 68520
  Number of trainable parameters = 173042160
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,4.226600
1000,3.832400
1500,3.550500
2000,3.450500
2500,3.313500
3000,3.260000
3500,3.197100
4000,3.129300
4500,3.097800
5000,3.040700


Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-10000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/checkpoint-20000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked

TrainOutput(global_step=68520, training_loss=3.448556746924334, metrics={'train_runtime': 25669.9352, 'train_samples_per_second': 85.408, 'train_steps_per_second': 2.669, 'total_flos': 2.41817448964608e+16, 'train_loss': 3.448556746924334, 'epoch': 10.0})

In [19]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
#trainer.save_model("./KantaiBERT")
trainer.save_model('./BR_BERTo-CENTENFolha-8_parts')

Saving model checkpoint to ./BR_BERTo-CENTENFolha-8_parts
Configuration saved in ./BR_BERTo-CENTENFolha-8_parts/config.json
Model weights saved in ./BR_BERTo-CENTENFolha-8_parts/pytorch_model.bin


In [20]:
# Saving model on Wandb
import wandb
wandb.save('BR_BERTo-CENTENFolha-8_parts.h5')

[]

In [21]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BR_BERTo-CENTENFolha-8_parts",
    tokenizer=tokenizer
)

loading configuration file ./BR_BERTo-CENTENFolha-8_parts/config.json
Model config RobertaConfig {
  "_name_or_path": "./BR_BERTo-CENTENFolha-8_parts",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 8,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 150000
}

loading configuration file ./BR_BERTo-CENTENFolha-8_parts/config.json
Model config RobertaConfig {
  "_name_or_path": "./BR_BERTo-CENTENFolha-8_parts",
  "architectures": [

In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 73252
  Batch size = 16


Perplexity: 60.33


In [23]:
eval_results

{'eval_loss': 4.099853038787842,
 'eval_runtime': 484.5416,
 'eval_samples_per_second': 151.178,
 'eval_steps_per_second': 9.45,
 'epoch': 10.0}

In [24]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.38669583201408386,
  'token': 225,
  'token_str': ' ',
  'sequence': 'O rapaz olhou para o  '},
 {'score': 0.24862034618854523,
  'token': 18,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o. '},
 {'score': 0.10848167538642883,
  'token': 35,
  'token_str': '?',
  'sequence': 'O rapaz olhou para o? '},
 {'score': 0.021669907495379448,
  'token': 5,
  'token_str': '!',
  'sequence': 'O rapaz olhou para o! '},
 {'score': 0.010248017497360706,
  'token': 23430,
  'token_str': '»',
  'sequence': 'O rapaz olhou para o» '}]

In [25]:
fill_mask("A moça olhou para o <mask> ")

[{'score': 0.378112256526947,
  'token': 225,
  'token_str': ' ',
  'sequence': 'A moça olhou para o  '},
 {'score': 0.24172565340995789,
  'token': 18,
  'token_str': '.',
  'sequence': 'A moça olhou para o. '},
 {'score': 0.10514548420906067,
  'token': 35,
  'token_str': '?',
  'sequence': 'A moça olhou para o? '},
 {'score': 0.01929977536201477,
  'token': 5,
  'token_str': '!',
  'sequence': 'A moça olhou para o! '},
 {'score': 0.010475827381014824,
  'token': 23430,
  'token_str': '»',
  'sequence': 'A moça olhou para o» '}]

In [26]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.34191733598709106,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Comprou uma  na loja. '},
 {'score': 0.08656112104654312,
  'token': 3404,
  'token_str': ' loja',
  'sequence': 'Comprou uma loja na loja. '},
 {'score': 0.025564001873135567,
  'token': 1933,
  'token_str': ' foto',
  'sequence': 'Comprou uma foto na loja. '},
 {'score': 0.02493678219616413,
  'token': 4194,
  'token_str': ' peça',
  'sequence': 'Comprou uma peça na loja. '},
 {'score': 0.01081919576972723,
  'token': 3117,
  'token_str': ' venda',
  'sequence': 'Comprou uma venda na loja. '}]

In [27]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.9603817462921143,
  'token': 225,
  'token_str': ' ',
  'sequence': 'A mulher não é. '},
 {'score': 0.015548727475106716,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'A mulher não é «. '},
 {'score': 0.0008894373313523829,
  'token': 23430,
  'token_str': '»',
  'sequence': 'A mulher não é». '},
 {'score': 0.0007877573953010142,
  'token': 35,
  'token_str': '?',
  'sequence': 'A mulher não é?. '},
 {'score': 0.0006621446809731424,
  'token': 18,
  'token_str': '.',
  'sequence': 'A mulher não é.. '}]

In [28]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.9436227679252625,
  'token': 225,
  'token_str': ' ',
  'sequence': 'O homem não é. '},
 {'score': 0.024407250806689262,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'O homem não é «. '},
 {'score': 0.00109079759567976,
  'token': 35,
  'token_str': '?',
  'sequence': 'O homem não é?. '},
 {'score': 0.0010504191741347313,
  'token': 23430,
  'token_str': '»',
  'sequence': 'O homem não é». '},
 {'score': 0.0010156186763197184,
  'token': 18,
  'token_str': '.',
  'sequence': 'O homem não é.. '}]

In [29]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.9600668549537659,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Ele é um bom. '},
 {'score': 0.0231776162981987,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Ele é um bom «. '},
 {'score': 0.001503376173786819,
  'token': 23430,
  'token_str': '»',
  'sequence': 'Ele é um bom». '},
 {'score': 0.00033725512912496924,
  'token': 16,
  'token_str': ',',
  'sequence': 'Ele é um bom,. '},
 {'score': 0.0003097472945228219,
  'token': 373,
  'token_str': ' não',
  'sequence': 'Ele é um bom não. '}]

In [30]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.9358880519866943,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Ela é uma boa. '},
 {'score': 0.02805684693157673,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Ela é uma boa «. '},
 {'score': 0.0021303596440702677,
  'token': 1663,
  'token_str': ' coisa',
  'sequence': 'Ela é uma boa coisa. '},
 {'score': 0.0019621001556515694,
  'token': 23430,
  'token_str': '»',
  'sequence': 'Ela é uma boa». '},
 {'score': 0.00042498187394812703,
  'token': 1955,
  'token_str': ' música',
  'sequence': 'Ela é uma boa música. '}]

In [31]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.936142086982727,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Faz de conta que ainda é. '},
 {'score': 0.006818974856287241,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Faz de conta que ainda é «. '},
 {'score': 0.0019941481295973063,
  'token': 1208,
  'token_str': ' bom',
  'sequence': 'Faz de conta que ainda é bom. '},
 {'score': 0.0016038106987252831,
  'token': 373,
  'token_str': ' não',
  'sequence': 'Faz de conta que ainda é não. '},
 {'score': 0.001420637359842658,
  'token': 1428,
  'token_str': ' nada',
  'sequence': 'Faz de conta que ainda é nada. '}]

In [32]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.7687026858329773,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Depois da tempestade vem a. '},
 {'score': 0.01661275327205658,
  'token': 5244,
  'token_str': ' chuva',
  'sequence': 'Depois da tempestade vem a chuva. '},
 {'score': 0.010111875832080841,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Depois da tempestade vem a «. '},
 {'score': 0.00932687520980835,
  'token': 1424,
  'token_str': ' água',
  'sequence': 'Depois da tempestade vem a água. '},
 {'score': 0.0041083223186433315,
  'token': 3656,
  'token_str': ' temperatura',
  'sequence': 'Depois da tempestade vem a temperatura. '}]

In [33]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.18515491485595703,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Mais vale um pássaro na mão do que  voando. '},
 {'score': 0.04191816225647926,
  'token': 405,
  'token_str': ' mais',
  'sequence': 'Mais vale um pássaro na mão do que mais voando. '},
 {'score': 0.032877855002880096,
  'token': 603,
  'token_str': ' está',
  'sequence': 'Mais vale um pássaro na mão do que está voando. '},
 {'score': 0.029668239876627922,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Mais vale um pássaro na mão do que « voando. '},
 {'score': 0.027036333456635475,
  'token': 1021,
  'token_str': ' seja',
  'sequence': 'Mais vale um pássaro na mão do que seja voando. '}]

In [34]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.10223205387592316,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Não tinha, mas almoçou mesmo assim. '},
 {'score': 0.06879472732543945,
  'token': 1428,
  'token_str': ' nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim. '},
 {'score': 0.013728772290050983,
  'token': 702,
  'token_str': ' isso',
  'sequence': 'Não tinha isso, mas almoçou mesmo assim. '},
 {'score': 0.013108884915709496,
  'token': 942,
  'token_str': ' tudo',
  'sequence': 'Não tinha tudo, mas almoçou mesmo assim. '},
 {'score': 0.01261900831013918,
  'token': 1918,
  'token_str': ' problema',
  'sequence': 'Não tinha problema, mas almoçou mesmo assim. '}]

In [35]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.9809194207191467,
  'token': 225,
  'token_str': ' ',
  'sequence': "Bebeu um copo d'água, pois tinha. "},
 {'score': 0.006993266753852367,
  'token': 3639,
  'token_str': ' «',
  'sequence': "Bebeu um copo d'água, pois tinha «. "},
 {'score': 0.0009519975865259767,
  'token': 23430,
  'token_str': '»',
  'sequence': "Bebeu um copo d'água, pois tinha». "},
 {'score': 0.0003567138919606805,
  'token': 18,
  'token_str': '.',
  'sequence': "Bebeu um copo d'água, pois tinha.. "},
 {'score': 0.0002893344790209085,
  'token': 5,
  'token_str': '!',
  'sequence': "Bebeu um copo d'água, pois tinha!. "}]

In [36]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.862694263458252,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Sem saber, ele descobriu um. '},
 {'score': 0.029595544561743736,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Sem saber, ele descobriu um «. '},
 {'score': 0.006179000250995159,
  'token': 1139,
  'token_str': ' outro',
  'sequence': 'Sem saber, ele descobriu um outro. '},
 {'score': 0.0057591102086007595,
  'token': 1449,
  'token_str': ' tipo',
  'sequence': 'Sem saber, ele descobriu um tipo. '},
 {'score': 0.002536940388381481,
  'token': 1309,
  'token_str': ' pouco',
  'sequence': 'Sem saber, ele descobriu um pouco. '}]

In [37]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.9651967287063599,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Pedro fez fortuna vendendo. '},
 {'score': 0.010250731371343136,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Pedro fez fortuna vendendo «. '},
 {'score': 0.003451635129749775,
  'token': 23430,
  'token_str': '»',
  'sequence': 'Pedro fez fortuna vendendo». '},
 {'score': 0.000461291492683813,
  'token': 17,
  'token_str': '-',
  'sequence': 'Pedro fez fortuna vendendo-. '},
 {'score': 0.0004227626195643097,
  'token': 16,
  'token_str': ',',
  'sequence': 'Pedro fez fortuna vendendo,. '}]

In [38]:
fill_mask("Sem medo de ser <mask>. ")

[{'score': 0.9583428502082825,
  'token': 225,
  'token_str': ' ',
  'sequence': 'Sem medo de ser. '},
 {'score': 0.006450172979384661,
  'token': 3639,
  'token_str': ' «',
  'sequence': 'Sem medo de ser «. '},
 {'score': 0.0016045953379943967,
  'token': 315,
  'token_str': ' um',
  'sequence': 'Sem medo de ser um. '},
 {'score': 0.0010848623933270574,
  'token': 268,
  'token_str': ' de',
  'sequence': 'Sem medo de ser de. '},
 {'score': 0.0006845065508969128,
  'token': 1208,
  'token_str': ' bom',
  'sequence': 'Sem medo de ser bom. '}]